In [168]:
import requests
from bs4 import BeautifulSoup, NavigableString
import math
import Levenshtein
from langdetect import detect

In [175]:
def getQuotesByAuthor(author, maxChars, page_num = None):
    old_author = author
    author = author.replace(" ", "+")
    all_quotes = []

    # if page number not specified, get true page number
    if page_num is None:
        try:
            page = requests.get("https://www.goodreads.com/quotes/search??utf8=%E2%9C%93&q=" + author + "&commit=Search")
            soup = BeautifulSoup(page.text, 'html.parser')
            pages = soup.find(class_="smallText").text
            of = pages.find("of ")
            showing = pages.find("Showing ")

            num_shown = pages[showing+10:of-1]
            total_num = pages[of+3:]
            total_num = total_num.replace(",", "").replace("\n", "")
            num_shown = int(num_shown)
            total_num = int(total_num)
            page_num = math.ceil(total_num/num_shown)
            print("looking through", page_num, "pages")
        except:
            page_num = 1
    
    # for each page
    for i in range(1, page_num+1, 1):
        try:
            page = requests.get("https://www.goodreads.com/quotes/search?commit=Search&page=" + str(i) + "&q=" + author + "&utf8=%E2%9C%93")
            soup = BeautifulSoup(page.text, 'html.parser')
            print("scraping page", i, " of ", page_num)
        except:
            print("could not connect to goodreads")
            break

        try:
            quote = soup.find(class_="leftContainer")
            quote_list = quote.find_all(class_="quoteDetails")
        except:
            pass

        # get data for each quote
        for quote in quote_list:

            meta_data = []
            # Get quote's author
            try:
                qAuthor = quote.find(class_="authorOrTitle").text
                qAuthor = qAuthor.replace(",", "")
                qAuthor = qAuthor.replace("\n", "")
                qAuthor = qAuthor.strip()
                qAuthor = qAuthor.replace(" ", "+")
                
            except:
                meta_data.append(None)

            #if its a quote from a diff author
            if(Levenshtein.distance(qAuthor,author) > 5): 
                continue
            else:
                meta_data.append(qAuthor.replace("+"," ").strip())

            # Get quote's text
            try:
                outer = quote.find(class_="quoteText")
                inner_text = [element for element in outer if isinstance(element, NavigableString)]
                final_quote = "\n".join(inner_text[:])
                final_quote = final_quote.replace("\n", " ")
                final_quote = final_quote.replace("―", "").strip()
                
            except:
                pass 

            if(len(final_quote) < maxChars and len(final_quote) != 0 and detect(final_quote) == "en"):
                    meta_data.append(final_quote)
            else:
                meta_data.append(None)
                continue
            # Get quote's book title
            try: 
                title = quote.find(class_="authorOrTitle")
                title = title.nextSibling.nextSibling.text
                # title = title.replace("\n", "")
                meta_data.append(title.strip())
                # print(title)
            except:
                meta_data.append(None)

            # Get quote's tags
            try:
                tags = quote.find(class_="greyText smallText left").text
                tags = [x.strip() for x in tags.split(',')]
                tags = tags[1:]
                meta_data.append(tags)
                # print(tags)
            except:
                meta_data.append(None)

            # Get number of likes
            try:
                likes = quote.find(class_="right").text
                likes = likes.replace("likes", "")
                likes = int(likes)
                meta_data.append(likes)
                # print(likes)
            except:
                meta_data.append(None)

            all_quotes.append(meta_data)

    #for text, author, title, tags, likes in all_quotes:
    #    print(text)
    #    print(author)
    #    print(title)
    #    print(tags)
    #    print(likes)
    #    print()

    return all_quotes


In [176]:
Quotes = getQuotesByAuthor("William Shakespeare",175,20)

scraping page 1  of  20
scraping page 2  of  20
scraping page 3  of  20
scraping page 4  of  20
scraping page 5  of  20
scraping page 6  of  20
scraping page 7  of  20
scraping page 8  of  20
scraping page 9  of  20
scraping page 10  of  20
scraping page 11  of  20
scraping page 12  of  20
scraping page 13  of  20
scraping page 14  of  20
scraping page 15  of  20
scraping page 16  of  20
scraping page 17  of  20
scraping page 18  of  20
scraping page 19  of  20
scraping page 20  of  20


In [180]:
for i in range(0,len(Quotes)):
    print(Quotes[i][1])

“The fool doth think he is wise, but the wise man knows himself to be a fool.”
“Love all, trust a few, do wrong to none.”
“Love looks not with the eyes, but with the mind, And therefore is winged Cupid painted blind.”
“Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”
“Doubt thou the stars are fire; Doubt that the sun doth move; Doubt truth to be a liar; But never doubt I love.”
“The fault, dear Brutus, is not in our stars, but in ourselves.”
“This above all: to thine own self be true,  And it must follow, as the night the day,  Thou canst not then be false to any man.”
“There is nothing either good or bad, but thinking makes it so.”
“Hell is empty and all the devils are here.”
“It is not in the stars to hold our destiny but in ourselves.”
“If music be the food of love, play on, Give me excess of it; that surfeiting, The appetite may sicken, and so die.”
“We know what we are, but not what we may be.”
“Words are easy, l

In [137]:
all_quotes = []
author = "William Shakespeare"
old_author = author
page_num = 1
author = author.replace(" ", "+")

page = requests.get("https://www.goodreads.com/quotes/search??utf8=%E2%9C%93&q=" + author + "&commit=Search")
soup = BeautifulSoup(page.text, 'html.parser')
pages = soup.find(class_="smallText").text
of = pages.find("of ")
showing = pages.find("Showing ")

num_shown = pages[showing+10:of-1]
total_num = pages[of+3:]
total_num = total_num.replace(",", "").replace("\n", "")
num_shown = int(num_shown)
total_num = int(total_num)
page_num = math.ceil(total_num/num_shown)
print("looking through", page_num, "pages")

looking through 308 pages


In [156]:
for i in range (1,2):
    try:
        page = requests.get("https://www.goodreads.com/quotes/search?commit=Search&page=" + str(i) + "&q=" + author + "&utf8=%E2%9C%93")
        soup = BeautifulSoup(page.text, 'html.parser')
        print("scraping page", i)
    except:
        print("could not connect to goodreads")

    try:
        quote = soup.find(class_="leftContainer")
        quote_list = quote.find_all(class_="quoteDetails")
    except:
        pass

        # get data for each quote
    for quote in quote_list:

        meta_data = []
                # Get quote's author
        try:
            qAuthor = quote.find(class_="authorOrTitle").text
            qAuthor = qAuthor.replace(",", "")
            qAuthor = qAuthor.replace("\n", "")
            qAuthor = qAuthor.strip()
            qAuthor = qAuthor.replace(" ", "+")

            if(Levenshtein.distance(qAuthor,author) > 5):
                continue
                    
            else:
                meta_data.append(author.strip())
                    # print(author)
        except:
            meta_data.append(None)

        try:
            outer = quote.find(class_="quoteText")
            inner_text = [element for element in outer if isinstance(element, NavigableString)]
            final_quote = "\n".join(inner_text[:])
            final_quote = final_quote.replace("\n", " ")
            final_quote = final_quote.replace("―", "").strip()

            meta_data.append(final_quote)
            if(len(final_quote) < maxWords):
                print(inner_text)
                print(final_quote)

                meta_data.append(final_quote)
            else:
                continue
        except:
            meta_data.append("FAIL")
            pass
        

        all_quotes.append(meta_data)




scraping page 1
[&#39;\n      “The fool doth think he is wise, but the wise man knows himself to be a fool.”\n  &#39;, &#39;  ―\n  &#39;, &#39;\n&#39;, &#39;\n&#39;, &#39;\n&#39;, &#39;\n&#39;]
“The fool doth think he is wise, but the wise man knows himself to be a fool.”
[&#39;\n      “Love all, trust a few, do wrong to none.”\n  &#39;, &#39;  ―\n  &#39;, &#39;\n&#39;, &#39;\n&#39;, &#39;\n&#39;]
“Love all, trust a few, do wrong to none.”
[&#39;\n      “Love looks not with the eyes, but with the mind,&#39;, &#39;And therefore is winged Cupid painted blind.”\n  &#39;, &#39;  ―\n  &#39;, &#39;\n&#39;, &#39;\n&#39;, &#39;\n&#39;]
“Love looks not with the eyes, but with the mind, And therefore is winged Cupid painted blind.”
[&#39;\n      “Be not afraid of greatness. Some are born great, some achieve greatness, and others have greatness thrust upon them.”\n  &#39;, &#39;  ―\n  &#39;, &#39;\n&#39;, &#39;\n&#39;, &#39;\n&#39;]
“Be not afraid of greatness. Some are born great, some achieve g

In [167]:
detect("hello")

NameError: name &#39;detect&#39; is not defined